# Naive Linear Solver - Prototype

Greedy row-by-row puzzle solver implementation using existing grid/scoring infrastructure.


## Import


In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from loguru import logger as lg
from rich import get_console
from rich import print as rprint
from rich.console import Console

# some magic to make rich work in jupyter
# https://github.com/Textualize/rich/issues/3483
# enable it for every cell output with %load_ext rich
console: Console = get_console()
console.is_jupyter = False

In [ ]:
import random
from functools import partial

from snap_fit.config.aruco.aruco_board_config import ArucoBoardConfig
from snap_fit.config.aruco.aruco_detector_config import ArucoDetectorConfig
from snap_fit.data_models.piece_id import PieceId
from snap_fit.grid.grid_model import GridModel
from snap_fit.grid.orientation import Orientation, PieceType
from snap_fit.grid.placement_state import PlacementState
from snap_fit.grid.scoring import score_edge, score_grid, score_grid_with_details
from snap_fit.grid.types import GridPos
from snap_fit.params.snap_fit_params import get_snap_fit_paths
from snap_fit.puzzle.piece_matcher import PieceMatcher
from snap_fit.puzzle.sheet_aruco import SheetAruco
from snap_fit.puzzle.sheet_manager import SheetManager

## Load puzzle data


In [ ]:
# 1. Configure ArUco Board and Detector
board_config = ArucoBoardConfig(
    markers_x=7,
    markers_y=5,
    marker_length=100,
    marker_separation=100,
)
detector_config = ArucoDetectorConfig(board=board_config)

# 2. Initialize SheetAruco helper
sheet_aruco = SheetAruco(detector_config)
aruco_loader = partial(sheet_aruco.load_sheet, min_area=5_000)

# 3. Load sheets
paths = get_snap_fit_paths()
data_dir = paths.data_fol / "sample_puzzle_v1" / "sheets"
lg.info(f"Loading data from {data_dir}")

manager = SheetManager()
manager.add_sheets(
    folder_path=data_dir,
    pattern="*.png",
    loader_func=aruco_loader,
)

lg.info(
    f"Loaded {len(manager.get_sheets_ls())} sheets with {len(manager.get_pieces_ls())} pieces"
)

## Partition pieces by type


In [ ]:
def partition_pieces_by_type(
    manager: SheetManager,
) -> tuple[list[PieceId], list[PieceId], list[PieceId]]:
    """Partition all pieces by their type (corner, edge, inner).

    Returns:
        Tuple of (corners, edges, inners) as lists of PieceId.
    """
    corners: list[PieceId] = []
    edges: list[PieceId] = []
    inners: list[PieceId] = []

    for piece in manager.get_pieces_ls():
        piece_type = piece.oriented_piece_type.piece_type
        if piece_type == PieceType.CORNER:
            corners.append(piece.piece_id)
        elif piece_type == PieceType.EDGE:
            edges.append(piece.piece_id)
        else:
            inners.append(piece.piece_id)

    return corners, edges, inners


corners, edges, inners = partition_pieces_by_type(manager)
lg.info(
    f"Partitioned: {len(corners)} corners, {len(edges)} edges, {len(inners)} inners"
)

## Initialize matcher and pre-compute scores


In [ ]:
# Initialize matcher and pre-compute all pairwise scores
matcher = PieceMatcher(manager)
matcher.match_all()
lg.info(f"Cached {len(matcher.results)} segment pair scores")

## NaiveLinearSolver class


In [ ]:
ALL_ORIENTATIONS = [
    Orientation.DEG_0,
    Orientation.DEG_90,
    Orientation.DEG_180,
    Orientation.DEG_270,
]


class NaiveLinearSolver:
    """Greedy row-by-row puzzle solver.

    Places pieces starting from the top-left corner, filling each row
    left-to-right before moving to the next row. Uses a greedy strategy
    to select the best-matching piece at each position.

    Handles misclassified pieces gracefully by falling back to any available
    piece when the expected type runs out.
    """

    def __init__(
        self,
        grid: GridModel,
        matcher: PieceMatcher,
        corners: list[PieceId],
        edges: list[PieceId],
        inners: list[PieceId],
    ) -> None:
        """Initialize the solver.

        Args:
            grid: The grid model defining puzzle structure.
            matcher: Pre-populated piece matcher with cached scores.
            corners: Available corner piece IDs.
            edges: Available edge piece IDs.
            inners: Available inner piece IDs.
        """
        self.grid = grid
        self.matcher = matcher
        self.state = PlacementState(grid)

        # Mutable pools of available pieces
        self._corners = list(corners)
        self._edges = list(edges)
        self._inners = list(inners)

        # Track if we had to abort early
        self._aborted = False
        self._abort_reason = ""

    def _get_all_available(self) -> list[PieceId]:
        """Get all remaining available pieces regardless of type."""
        return self._corners + self._edges + self._inners

    def _remove_from_pool(self, piece_id: PieceId) -> None:
        """Remove a piece from whichever pool it belongs to."""
        if piece_id in self._corners:
            self._corners.remove(piece_id)
        elif piece_id in self._edges:
            self._edges.remove(piece_id)
        elif piece_id in self._inners:
            self._inners.remove(piece_id)

    def _get_candidates_with_fallback(
        self, primary: list[PieceId], slot_type: str
    ) -> list[PieceId]:
        """Get candidates, falling back to all available if primary is empty.

        Args:
            primary: The preferred candidate list (e.g., corners for corner slots).
            slot_type: Description for logging (e.g., "corner", "edge").

        Returns:
            List of candidate piece IDs.
        """
        if primary:
            return primary

        # Fallback: use any available piece
        all_available = self._get_all_available()
        if all_available:
            lg.warning(
                f"No {slot_type} pieces left, falling back to {len(all_available)} "
                f"remaining pieces of other types"
            )
            return all_available

        return []

    def solve(self) -> PlacementState:
        """Execute the greedy solver.

        Returns:
            PlacementState with pieces placed (may be incomplete if pieces run out).
        """
        lg.info(f"Starting solve for {self.grid.rows}x{self.grid.cols} grid")
        lg.info(
            f"Available: {len(self._corners)} corners, {len(self._edges)} edges, "
            f"{len(self._inners)} inners"
        )

        try:
            # Row 0
            self._place_row_zero()

            # Remaining rows
            for row in range(1, self.grid.rows):
                self._place_subsequent_row(row)

        except StopIteration as e:
            self._aborted = True
            self._abort_reason = str(e)
            lg.error(f"Solver aborted: {e}")

        lg.info(
            f"Solve complete. Placed {self.state.placed_count}/{self.grid.total_cells} pieces."
        )
        if self._aborted:
            lg.warning(f"Puzzle incomplete due to: {self._abort_reason}")

        return self.state

    def _place_row_zero(self) -> None:
        """Place the first row: corner → edges → corner."""
        # Top-left corner (0, 0) - random selection
        pos_tl = GridPos(ro=0, co=0)
        candidates = self._get_candidates_with_fallback(self._corners, "corner")
        if not candidates:
            raise StopIteration("No pieces available for top-left corner")

        corner_tl = random.choice(candidates)
        self._remove_from_pool(corner_tl)
        self.state.place(corner_tl, pos_tl, Orientation.DEG_0)
        lg.debug(f"Placed top-left corner {corner_tl} at {pos_tl}")

        # Edge pieces for columns 1..cols-2
        for col in range(1, self.grid.cols - 1):
            pos = GridPos(ro=0, co=col)
            required_orientation = self.grid.get_slot_type(pos).orientation
            neighbors = [GridPos(ro=0, co=col - 1)]  # left neighbor only

            candidates = self._get_candidates_with_fallback(self._edges, "edge")
            if not candidates:
                raise StopIteration(f"No pieces available for position {pos}")

            best_piece, best_orient, best_score = self._find_best_piece(
                candidates=candidates,
                pos=pos,
                neighbors=neighbors,
                orientations=[required_orientation],
            )
            self._remove_from_pool(best_piece)
            self.state.place(best_piece, pos, best_orient)
            lg.debug(f"Placed edge {best_piece} at {pos} with score {best_score:.4f}")

        # Top-right corner (0, cols-1)
        pos_tr = GridPos(ro=0, co=self.grid.cols - 1)
        required_orientation = self.grid.get_slot_type(pos_tr).orientation
        neighbors = [GridPos(ro=0, co=self.grid.cols - 2)]  # left neighbor

        candidates = self._get_candidates_with_fallback(self._corners, "corner")
        if not candidates:
            raise StopIteration(f"No pieces available for position {pos_tr}")

        best_piece, best_orient, best_score = self._find_best_piece(
            candidates=candidates,
            pos=pos_tr,
            neighbors=neighbors,
            orientations=[required_orientation],
        )
        self._remove_from_pool(best_piece)
        self.state.place(best_piece, pos_tr, best_orient)
        lg.debug(
            f"Placed top-right corner {best_piece} at {pos_tr} with score {best_score:.4f}"
        )

    def _place_subsequent_row(self, row: int) -> None:
        """Place a row using top and left neighbor constraints.

        Args:
            row: The row index (1 to rows-1).
        """
        is_last_row = row == self.grid.rows - 1

        # Left edge or bottom-left corner
        pos_left = GridPos(ro=row, co=0)
        required_orientation = self.grid.get_slot_type(pos_left).orientation
        neighbors = [GridPos(ro=row - 1, co=0)]  # top neighbor

        if is_last_row:
            candidates = self._get_candidates_with_fallback(self._corners, "corner")
        else:
            candidates = self._get_candidates_with_fallback(self._edges, "edge")

        if not candidates:
            raise StopIteration(f"No pieces available for position {pos_left}")

        best_piece, best_orient, best_score = self._find_best_piece(
            candidates=candidates,
            pos=pos_left,
            neighbors=neighbors,
            orientations=[required_orientation],
        )
        self._remove_from_pool(best_piece)
        self.state.place(best_piece, pos_left, best_orient)
        lg.debug(
            f"Placed left piece {best_piece} at {pos_left} with score {best_score:.4f}"
        )

        # Inner pieces (or bottom edges for last row) for columns 1..cols-2
        for col in range(1, self.grid.cols - 1):
            pos = GridPos(ro=row, co=col)
            neighbors = [
                GridPos(ro=row, co=col - 1),  # left
                GridPos(ro=row - 1, co=col),  # top
            ]

            if is_last_row:
                required_orientation = self.grid.get_slot_type(pos).orientation
                candidates = self._get_candidates_with_fallback(self._edges, "edge")
                orientations = [required_orientation]
            else:
                candidates = self._get_candidates_with_fallback(self._inners, "inner")
                orientations = ALL_ORIENTATIONS

            if not candidates:
                raise StopIteration(f"No pieces available for position {pos}")

            best_piece, best_orient, best_score = self._find_best_piece(
                candidates=candidates,
                pos=pos,
                neighbors=neighbors,
                orientations=orientations,
            )
            self._remove_from_pool(best_piece)
            self.state.place(best_piece, pos, best_orient)
            lg.debug(
                f"Placed {best_piece} at {pos} orient={best_orient} score={best_score:.4f}"
            )

        # Right edge or bottom-right corner
        pos_right = GridPos(ro=row, co=self.grid.cols - 1)
        required_orientation = self.grid.get_slot_type(pos_right).orientation
        neighbors = [
            GridPos(ro=row, co=self.grid.cols - 2),  # left
            GridPos(ro=row - 1, co=self.grid.cols - 1),  # top
        ]

        if is_last_row:
            candidates = self._get_candidates_with_fallback(self._corners, "corner")
        else:
            candidates = self._get_candidates_with_fallback(self._edges, "edge")

        if not candidates:
            raise StopIteration(f"No pieces available for position {pos_right}")

        best_piece, best_orient, best_score = self._find_best_piece(
            candidates=candidates,
            pos=pos_right,
            neighbors=neighbors,
            orientations=[required_orientation],
        )
        self._remove_from_pool(best_piece)
        self.state.place(best_piece, pos_right, best_orient)
        lg.debug(
            f"Placed right piece {best_piece} at {pos_right} with score {best_score:.4f}"
        )

    def _find_best_piece(
        self,
        candidates: list[PieceId],
        pos: GridPos,
        neighbors: list[GridPos],
        orientations: list[Orientation],
    ) -> tuple[PieceId, Orientation, float]:
        """Score all candidates and orientations against placed neighbors.

        Args:
            candidates: Available pieces to evaluate.
            pos: Target grid position.
            neighbors: Adjacent positions with already-placed pieces.
            orientations: List of orientations to try for each candidate.

        Returns:
            Tuple of (best_piece_id, best_orientation, best_score).

        Raises:
            StopIteration: If no candidates available.
        """
        if not candidates:
            raise StopIteration(f"No candidates available for position {pos}")

        best_piece: PieceId | None = None
        best_orient: Orientation = Orientation.DEG_0
        best_score = float("inf")

        for piece_id in candidates:
            for orientation in orientations:
                # Temporarily place to compute score
                self.state.place(piece_id, pos, orientation)

                # Sum scores against all placed neighbors
                total_score = 0.0
                for neighbor_pos in neighbors:
                    edge_score = score_edge(self.state, pos, neighbor_pos, self.matcher)
                    if edge_score is not None:
                        total_score += edge_score

                # Track best
                if total_score < best_score:
                    best_score = total_score
                    best_piece = piece_id
                    best_orient = orientation

                # Remove temporary placement
                self.state.remove(pos)

        if best_piece is None:
            raise StopIteration(f"Could not find best piece for position {pos}")

        return best_piece, best_orient, best_score

## Run the solver


In [ ]:
# Determine grid size from piece counts
# For an NxM puzzle: 4 corners, 2*(N-2) + 2*(M-2) edges, (N-2)*(M-2) inners
total_pieces = len(corners) + len(edges) + len(inners)
lg.info(f"Total pieces: {total_pieces}")
lg.info(f"Corners: {len(corners)}, Edges: {len(edges)}, Inners: {len(inners)}")

# For puzzles with possible mis-detected corners, we can infer from total pieces
# rows * cols = total_pieces
# Try common sizes
possible_sizes = [
    (3, 3),
    (3, 4),
    (4, 3),
    (4, 4),
    (4, 5),
    (5, 4),
    (5, 5),
    (4, 6),
    (6, 4),
    (5, 6),
    (6, 5),
    (6, 6),
    (6, 8),
    (8, 6),
    (7, 7),
    (8, 8),
]
detected_size = None

for rows, cols in possible_sizes:
    if rows * cols != total_pieces:
        continue

    expected_corners = 4
    expected_edges = 2 * (rows - 2) + 2 * (cols - 2)
    expected_inners = (rows - 2) * (cols - 2)

    # Allow some tolerance for mis-detected pieces
    # (corner could be detected as edge due to image quality)
    corner_diff = abs(len(corners) - expected_corners)
    edge_diff = abs(len(edges) - expected_edges)
    inner_diff = abs(len(inners) - expected_inners)

    if corner_diff + edge_diff + inner_diff <= 2:  # Allow up to 2 misdetections
        detected_size = (rows, cols)
        lg.info(
            f"Matched {rows}x{cols}: expected ({expected_corners}, {expected_edges}, {expected_inners})"
        )
        break

if detected_size:
    rows, cols = detected_size
    lg.info(f"Detected grid size: {rows}x{cols}")
else:
    # Last resort: find any grid that matches total_pieces
    for rows, cols in possible_sizes:
        if rows * cols == total_pieces:
            detected_size = (rows, cols)
            lg.warning(f"Using grid size {rows}x{cols} based on piece count only")
            break

    if detected_size:
        rows, cols = detected_size
    else:
        rows, cols = 4, 4
        lg.warning(f"Could not detect grid size, using default {rows}x{cols}")

# Create grid model
grid = GridModel(rows=rows, cols=cols)
rprint(grid)

In [ ]:
# Initialize and run solver
solver = NaiveLinearSolver(
    grid=grid,
    matcher=matcher,
    corners=corners,
    edges=edges,
    inners=inners,
)

result_state = solver.solve()

## Evaluate results


In [ ]:
# Compute total grid score
total_score, edge_scores = score_grid_with_details(result_state, matcher)

rprint(f"[bold green]Solver Results[/bold green]")
rprint(f"  Pieces placed: {result_state.placed_count}/{grid.total_cells}")
rprint(f"  Total score: {total_score:.4f} (lower is better)")
rprint(f"  Edges scored: {len(edge_scores)}/{grid.total_edges}")

In [ ]:
# Display the placement grid
rprint("\n[bold blue]Placement Grid[/bold blue]")
for ro in range(grid.rows):
    row_str = ""
    for co in range(grid.cols):
        pos = GridPos(ro=ro, co=co)
        placement = result_state.get_placement(pos)
        if placement:
            piece_id, orientation = placement
            # Short form: sheet_id:piece_id @ orientation
            orient_str = f"{orientation.value:>3}°"
            row_str += f"[{piece_id.piece_id:>2}@{orient_str}] "
        else:
            row_str += "[  empty  ] "
    rprint(row_str)

In [ ]:
# Show edge score statistics
scores = list(edge_scores.values())
if scores:
    rprint("\n[bold blue]Edge Score Statistics[/bold blue]")
    rprint(f"  Min score: {min(scores):.4f}")
    rprint(f"  Max score: {max(scores):.4f}")
    rprint(f"  Mean score: {sum(scores) / len(scores):.4f}")

    # Show worst edges (potential misplacements)
    sorted_edges = sorted(edge_scores.items(), key=lambda x: x[1], reverse=True)
    rprint("\n[bold yellow]Worst 5 edges (potential misplacements):[/bold yellow]")
    for (pos1, pos2), score in sorted_edges[:5]:
        p1 = result_state.get_placement(pos1)
        p2 = result_state.get_placement(pos2)
        if p1 and p2:
            rprint(f"  {pos1} <-> {pos2}: {score:.4f}")
            rprint(f"    {p1[0]} @ {p1[1].value}° <-> {p2[0]} @ {p2[1].value}°")

## Visualize solved puzzle


In [ ]:
import cv2
import numpy as np
from snap_fit.image.utils import show_image_mpl


def render_solved_puzzle(
    state: PlacementState,
    manager: SheetManager,
    cell_size: int = 150,
) -> np.ndarray:
    """Render the solved puzzle as a composite image.

    Args:
        state: The placement state with all pieces placed.
        manager: SheetManager to retrieve piece images.
        cell_size: Size of each cell in the output image.

    Returns:
        Composite image showing the puzzle.
    """
    grid = state.grid
    img_h = grid.rows * cell_size
    img_w = grid.cols * cell_size
    canvas = np.zeros((img_h, img_w, 3), dtype=np.uint8)

    for ro in range(grid.rows):
        for co in range(grid.cols):
            pos = GridPos(ro=ro, co=co)
            placement = state.get_placement(pos)
            if placement is None:
                continue

            piece_id, orientation = placement
            piece = manager.get_piece(piece_id)
            if piece is None:
                continue

            # Get piece image and rotate according to orientation
            piece_img = piece.img_orig.copy()

            # Rotate based on orientation (cv2 rotates counter-clockwise)
            if orientation == Orientation.DEG_90:
                piece_img = cv2.rotate(piece_img, cv2.ROTATE_90_CLOCKWISE)
            elif orientation == Orientation.DEG_180:
                piece_img = cv2.rotate(piece_img, cv2.ROTATE_180)
            elif orientation == Orientation.DEG_270:
                piece_img = cv2.rotate(piece_img, cv2.ROTATE_90_COUNTERCLOCKWISE)

            # Resize to fit cell
            piece_resized = cv2.resize(piece_img, (cell_size, cell_size))

            # Place in canvas
            y_start = ro * cell_size
            x_start = co * cell_size
            canvas[y_start : y_start + cell_size, x_start : x_start + cell_size] = (
                piece_resized
            )

    return canvas


# Render and display
puzzle_img = render_solved_puzzle(result_state, manager, cell_size=200)
show_image_mpl(puzzle_img, figsize=(12, 12))